<a id='importing-dependencies'></a>
<font size="+2" color='#053c96'><b> Introduction</b></font>  

Financial data is inherently sensitive, and even small adjustments can have significant implications. In this project, we have made a deliberate decision not to clean or manipulate the data beyond essential analysis steps. This approach ensures:

1. **Integrity of Financial Information**: 
   - Avoids introducing any errors or biases that could distort the original dataset.
   - Preserves the authenticity of the data for accurate insights.

2. **Ethical Considerations**: 
   - Prevents the project team from inadvertently altering the data in a way that could resemble fraudulent activity.

3. **Real-World Alignment**: 
   - Reflects the challenges analysts face when working with messy or incomplete financial data.
   - Encourages the development of robust methodologies to work with raw datasets.

While the dataset may contain inconsistencies, missing values, or other imperfections, these aspects are integral to the analysis process and are treated as part of the real-world financial landscape. This ensures transparency and ethical handling of data throughout the project.


<a id='importing-dependencies'></a>
<font size="+2" color='#053c96'><b> Importing Libraries</b></font>  

In [1]:
import sys
# Insert the parent path relative to this notebook so we can import from the src folder.
sys.path.insert(0, "..")

from src.dependencies import *
from src.functions import *

To import the necessary dependencies from the src folder, we have inserted the parent path relative to our notebook using sys.path.insert(0, "..").

<a id='data'></a>
<font size="+2" color='#053c96'><b> Data</b></font>  

#### Loading the data

Below is the data that will be used in model development

In [2]:
df = load_dataset()

The load_dataset() function reads a CSV file called financials_data.csv from a folder called data which is located in a directory above the current working directory using the relative path ../src/data/. It loads the data into a pandas DataFrame object named df and returns it.

In [3]:
# Removing 'PricingFinancials' from column names
df.columns = df.columns.str.replace('PricingFinancials', '')

In [4]:
df['Date'] = pd.to_datetime(df['FinancialsDate'])
df['Year'] = pd.to_datetime(df['FinancialsDate']).dt.year
df['Month'] = pd.to_datetime(df['FinancialsDate']).dt.month
df['Week'] = pd.to_datetime(df['FinancialsDate']).dt.isocalendar().week
df['Day'] = pd.to_datetime(df['FinancialsDate']).dt.day

In [5]:
# Drop categorical columns
cat_data = df.loc[:, ['Financial_Type', 'Industry', 'Country', 'FinancialsDate', 'Default', 'Date', 
                      'Year', 'Month', 'Week', 'Day']]
df.drop(['Financial_Type', 'Industry', 'Country', 'FinancialsDate', 'Default', 'Date', 'Year', 'Month', 'Week', 'Day'],
        inplace=True, axis=1)

In [6]:
# Find the count and percentage of zero-values
total_zeros = df.isin([0]).sum().sort_values(ascending=False)
percent_zeros = (df.isin([0]).sum()/df.isin([0]).count() * 100).sort_values(ascending=False)
df_zeros = pd.concat([total_zeros, percent_zeros], axis=1, keys=['Total Zeros', 'Percent Zeros'])

# Find a reasonable threshold for zero-values issues
test_zeros_level = 0.5
print(df_zeros.quantile(test_zeros_level))
_, thresh_zeros = df_zeros.quantile(test_zeros_level)

Total Zeros     4031.00
Percent Zeros     87.92
Name: 0.5, dtype: float64


The following code performs an analysis of zero-values in the dataset to identify potential issues with sparsity:

1. **Count and Percentage of Zero-Values**: 
   The total count and percentage of zero-values are calculated for each column.

2. **Determine Threshold for Zero-Value Issues**: 
   A reasonable threshold is identified based on the median (50th percentile) of zero-value metrics.


In [7]:
# Clean the dataset by applying thresholds for the zero values
print(f'INITIAL NUMBER OF VARIABLES: {df.shape[1]}')
print()

df_test1 = df.drop((df_zeros[df_zeros['Percent Zeros'] > thresh_zeros]).index, axis=1)
print(f'NUMBER OF VARIABLES AFTER Zeros THRESHOLD {thresh_zeros:.2f}%: {df_test1.shape[1]}')

INITIAL NUMBER OF VARIABLES: 195

NUMBER OF VARIABLES AFTER Zeros THRESHOLD 87.92%: 98


### Handling Outliers
Analyzing the dataframe with describtive method earlier showed a huge discrepancy between maximum value and 75% quantile. It also showed huge standard deviation values which could be a sign of the presence of outliers.

Dropping the top 3% and bottom 3% of the data for each financial feature should be able to handle the outliers conservatively.

In [8]:
# Analyze dataframe
df_test1.describe()

,DimFacilityKey,ID,Revenue,CostOfSales,GrossProfit,OperatingExpensesOverheads,OtherGainsOrLosses,EBITDA,Depreciation,EBIT,FinanceIncome,OtherBeforeEBIT,FinanceCost,NetProfitBeforeTax,Tax,NetProfitAfterTax,TotalCostBase,DepreciationAmortisationTotal,EBITBeforeFairValueAdjustments,OtherAfterEBIT,NCA_PropertyPlantEquipment,NCA_Goodwill,NCA_OtherIntangibles,NCA_OtherNonCurrentAssets,NCA_TotalNonCurrentAssets,NCA_TotalLoansIssued,NCA_GoodwillAndOtherIntangibles,NCA_InvestmentsInAssociatesAndJVs,NCA_DeferredTaxAssets,CA_Inventories,CA_CashAndCashEquivalents,CA_TradeAndOtherReceivables,CA_TotalLoansIssued,CA_CurrentTaxationAssets,CA_OtherCurrentAssets,CA_TotalCurrentAssets,TotalAssets,CA_ContractsInProgress,NCL_DeferredTax,NCL_TotalLoansReceived,NCL_InstalmentSaleLiabilities,NCL_OtherNonCurrentLiabilities,NCL_TotalNonCurrentLiabilities,TotalLiabilities,NCL_TotalEquityAndLiabilities,NCL_OtherInterestBearingLoans,CL_TradeAndOtherPayables,CL_TaxationPayable,CL_BankOverdraft,CL_TotalLoansReceived,CL_ProvisionsForLiabilitiesChargesObligations,CL_InstalmentSaleLiabilities,CL_OtherCurrentLiabilities,CL_TotalCurrentLiabilities,CL_OtherInterestBearingLoans,CL_OtherFinancialLiabilities,StatedCapitalShareCapitalAndPremium,RetainedEarnings,DistributableReserves,TotalShareholderEquityBeforeMinorities,NonControllingInterests,TotalEquity,NetWorth,CFF_DividendsPaid,CFF_InterestBorrowingsRaised,CFF_InterestBorrowingsRepaid,CFF_IncDecLongTermLiabilities,CFF_NetIntegroupLoans,CFF_InstalmentsCapitalisedLeases,CFF_OtherOutflowsDebt,CFF_NetCFF,CFF_NetIncCashAndCashEquivalents,CFF_CashAtStartOfYear,CFF_CashAtEndOfYear,CFI_NetPurchasePPE,CFI_ProceedsOnDisposalOfPPE,CFI_IncDecOtherFinAssets,CFI_ProceedsLoanRepayments,CFI_NetCFI,CFO_CashFromOpsBeforeWorkingCapital,CFO_IncDecInventories,CFO_IncDecTradeAndOtherReceivables,CFO_IncDecTradeAndOtherPayables,CFO_OtherWorkingCapitalMovements,CFO_CashOperatingActivities,CFO_InterestPaid,CFO_InterestReceived,CFO_TaxationPaid,CFO_NetCFO,NCA_LoansToRelatedParties,CA_TradeReceivables,CA_OtherReceivables,CA_LoansToRelatedParties,CL_OtherProvisions,CL_TradePayables,CL_OtherPayables,CL_LoansFromRelatedParties,NCL_LoansFromRelatedParties
count,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00
mean,2666.44,4243.43,2144070665.61,-1500810732.58,642992743.68,-305698835.12,1359113.01,245102194.24,-37892679.29,198594179.88,9299143.25,2932996.02,-63675335.67,147829790.96,-46139405.04,107368197.65,-1883621654.79,-43344468.71,197284965.06,2197018.84,385180392.75,43972187.68,109793584.56,43949337.70,1171650464.00,263723009.37,171544054.74,104339063.15,32436289.60,150391355.22,212999360.09,463186869.97,179852252.83,3978468.15,271484550.97,1482286298.97,2671299529.87,119720710.41,45863361.36,470717310.58,8440962.35,19926386.43,595660018.36,1744908970.54,2671305008.62,440034461.73,487362769.73,10793720.35,9333207.04,234234678.77,33703257.49,2130804.73,162931606.49,1143175570.37,226978153.31,36392848.18,158998365.12,527745810.80,73434863.32,867813577.03,54725863.13,926337583.26,711700304.67,-33707682.83,211361558.74,-144456877.49,9617849.86,317840.46,-627527.35,-23413925.82,14081175.27,9492052.14,64781830454.35,64790433558.48,-46191658.67,4529729.89,-85915.31,-213214.07,-68634480.17,205053089.04,-8892841.11,14597967.15,

In [9]:
# Cut outliers
top_quantiles = df_test1.quantile(0.97)
outliers_top = (df_test1 > top_quantiles)

low_quantiles = df_test1.quantile(0.03)
outliers_low = (df_test1 < low_quantiles)

df_test1 = df_test1.mask(outliers_top, top_quantiles, axis=1)
df_test1 = df_test1.mask(outliers_low, low_quantiles, axis=1)

# Take a look at the dataframe post-outliers cut
df_test1.describe()

,DimFacilityKey,ID,Revenue,CostOfSales,GrossProfit,OperatingExpensesOverheads,OtherGainsOrLosses,EBITDA,Depreciation,EBIT,FinanceIncome,OtherBeforeEBIT,FinanceCost,NetProfitBeforeTax,Tax,NetProfitAfterTax,TotalCostBase,DepreciationAmortisationTotal,EBITBeforeFairValueAdjustments,OtherAfterEBIT,NCA_PropertyPlantEquipment,NCA_Goodwill,NCA_OtherIntangibles,NCA_OtherNonCurrentAssets,NCA_TotalNonCurrentAssets,NCA_TotalLoansIssued,NCA_GoodwillAndOtherIntangibles,NCA_InvestmentsInAssociatesAndJVs,NCA_DeferredTaxAssets,CA_Inventories,CA_CashAndCashEquivalents,CA_TradeAndOtherReceivables,CA_TotalLoansIssued,CA_CurrentTaxationAssets,CA_OtherCurrentAssets,CA_TotalCurrentAssets,TotalAssets,CA_ContractsInProgress,NCL_DeferredTax,NCL_TotalLoansReceived,NCL_InstalmentSaleLiabilities,NCL_OtherNonCurrentLiabilities,NCL_TotalNonCurrentLiabilities,TotalLiabilities,NCL_TotalEquityAndLiabilities,NCL_OtherInterestBearingLoans,CL_TradeAndOtherPayables,CL_TaxationPayable,CL_BankOverdraft,CL_TotalLoansReceived,CL_ProvisionsForLiabilitiesChargesObligations,CL_InstalmentSaleLiabilities,CL_OtherCurrentLiabilities,CL_TotalCurrentLiabilities,CL_OtherInterestBearingLoans,CL_OtherFinancialLiabilities,StatedCapitalShareCapitalAndPremium,RetainedEarnings,DistributableReserves,TotalShareholderEquityBeforeMinorities,NonControllingInterests,TotalEquity,NetWorth,CFF_DividendsPaid,CFF_InterestBorrowingsRaised,CFF_InterestBorrowingsRepaid,CFF_IncDecLongTermLiabilities,CFF_NetIntegroupLoans,CFF_InstalmentsCapitalisedLeases,CFF_OtherOutflowsDebt,CFF_NetCFF,CFF_NetIncCashAndCashEquivalents,CFF_CashAtStartOfYear,CFF_CashAtEndOfYear,CFI_NetPurchasePPE,CFI_ProceedsOnDisposalOfPPE,CFI_IncDecOtherFinAssets,CFI_ProceedsLoanRepayments,CFI_NetCFI,CFO_CashFromOpsBeforeWorkingCapital,CFO_IncDecInventories,CFO_IncDecTradeAndOtherReceivables,CFO_IncDecTradeAndOtherPayables,CFO_OtherWorkingCapitalMovements,CFO_CashOperatingActivities,CFO_InterestPaid,CFO_InterestReceived,CFO_TaxationPaid,CFO_NetCFO,NCA_LoansToRelatedParties,CA_TradeReceivables,CA_OtherReceivables,CA_LoansToRelatedParties,CL_OtherProvisions,CL_TradePayables,CL_OtherPayables,CL_LoansFromRelatedParties,NCL_LoansFromRelatedParties
count,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00,4585.00
mean,2674.18,4208.17,774144728.21,-496103222.05,206213919.36,-65001288.21,1196624.48,50683748.10,-7286169.31,39043082.55,3534104.34,528467.29,-10787934.97,32922429.39,-10978212.52,24117636.83,-719411929.48,-9133761.37,36079564.21,182900.07,120216570.45,2948441.71,3029574.60,5550701.86,268433598.63,4811306.03,36634068.93,10110905.14,6025605.46,36051100.55,80768810.64,159107990.09,4112217.93,1681097.03,3966254.43,415200598.24,728075493.38,23026120.70,10540058.18,59183775.81,1316920.41,3413605.06,107736891.83,471339822.11,728080972.13,43012184.30,153878874.09,3346539.53,3096610.35,17317694.48,5373577.16,679395.81,4278622.58,320514686.40,10979232.40,831324.23,69795689.12,87511877.16,1469162.36,246137053.08,1965859.35,254579724.79,204783356.73,-2718388.34,8903179.37,-9996806.14,-8082.83,-32774.49,-105653.67,-1911610.18,-7523502.59,906635.20,59751079.70,61494679.13,-11920831.31,2022479.26,-120031.39,104465.60,-21008961.80,48069775.79,-454320.29,-1058356.69,941248.23,-581604.89,32843895.93,-2622429.84,1378319.69,-7641379.86,

The following code removes outliers from a dataset by capping and flooring extreme values based on predefined quantiles. The resulting dataset is then summarized to observe the impact.

1. **Identify Outliers**: 
   - Outliers are defined as values exceeding the 97th percentile (upper bound) or below the 3rd percentile (lower bound).

2. **Replace Outliers**: 
   - Values above the 97th percentile are capped at the 97th percentile value.
   - Values below the 3rd percentile are floored at the 3rd percentile value.

3. **Summary Post-Processing**: 
   - A statistical summary of the dataset is generated to inspect the effect of outlier treatment.


#### Add back the categorical data
Adding back the categorical dataet dropped in order to use this dataset with ML algorithms. 

Finally, print the information about the dataset

In [10]:
# Add back the classification columns
df_new = df_test1.join(cat_data)

# Print information about dataset
df_new.info()
df_new.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4585 entries, 0 to 4584
Columns: 108 entries, DimFacilityKey to Day
dtypes: UInt32(1), datetime64[ns](1), float64(98), int64(4), object(4)
memory usage: 3.8+ MB


,DimFacilityKey,ID,Revenue,CostOfSales,GrossProfit,OperatingExpensesOverheads,OtherGainsOrLosses,EBITDA,Depreciation,EBIT,FinanceIncome,OtherBeforeEBIT,FinanceCost,NetProfitBeforeTax,Tax,NetProfitAfterTax,TotalCostBase,DepreciationAmortisationTotal,EBITBeforeFairValueAdjustments,OtherAfterEBIT,NCA_PropertyPlantEquipment,NCA_Goodwill,NCA_OtherIntangibles,NCA_OtherNonCurrentAssets,NCA_TotalNonCurrentAssets,NCA_TotalLoansIssued,NCA_GoodwillAndOtherIntangibles,NCA_InvestmentsInAssociatesAndJVs,NCA_DeferredTaxAssets,CA_Inventories,CA_CashAndCashEquivalents,CA_TradeAndOtherReceivables,CA_TotalLoansIssued,CA_CurrentTaxationAssets,CA_OtherCurrentAssets,CA_TotalCurrentAssets,TotalAssets,CA_ContractsInProgress,NCL_DeferredTax,NCL_TotalLoansReceived,NCL_InstalmentSaleLiabilities,NCL_OtherNonCurrentLiabilities,NCL_TotalNonCurrentLiabilities,TotalLiabilities,NCL_TotalEquityAndLiabilities,NCL_OtherInterestBearingLoans,CL_TradeAndOtherPayables,CL_TaxationPayable,CL_BankOverdraft,CL_TotalLoansReceived,CL_ProvisionsForLiabilitiesChargesObligations,CL_InstalmentSaleLiabilities,CL_OtherCurrentLiabilities,CL_TotalCurrentLiabilities,CL_OtherInterestBearingLoans,CL_OtherFinancialLiabilities,StatedCapitalShareCapitalAndPremium,RetainedEarnings,DistributableReserves,TotalShareholderEquityBeforeMinorities,NonControllingInterests,TotalEquity,NetWorth,CFF_DividendsPaid,CFF_InterestBorrowingsRaised,CFF_InterestBorrowingsRepaid,CFF_IncDecLongTermLiabilities,CFF_NetIntegroupLoans,CFF_InstalmentsCapitalisedLeases,CFF_OtherOutflowsDebt,CFF_NetCFF,CFF_NetIncCashAndCashEquivalents,CFF_CashAtStartOfYear,CFF_CashAtEndOfYear,CFI_NetPurchasePPE,CFI_ProceedsOnDisposalOfPPE,CFI_IncDecOtherFinAssets,CFI_ProceedsLoanRepayments,CFI_NetCFI,CFO_CashFromOpsBeforeWorkingCapital,CFO_IncDecInventories,CFO_IncDecTradeAndOtherReceivables,CFO_IncDecTradeAndOtherPayables,CFO_OtherWorkingCapitalMovements,CFO_CashOperatingActivities,CFO_InterestPaid,CFO_InterestReceived,CFO_TaxationPaid,CFO_NetCFO,NCA_LoansToRelatedParties,CA_TradeReceivables,CA_OtherReceivables,CA_LoansToRelatedParties,CL_OtherProvisions,CL_TradePayables,CL_OtherPayables,CL_LoansFromRelatedParties,NCL_LoansFromRelatedParties,Financial_Type,Industry,Country,FinancialsDate,Default,Date,Year,Month,Week,Day
0,1664.00,179.56,19804960.00,-13696673.00,6108287.00,-5681818.00,677032.00,1103501.00,0.00,1105879.00,0.00,0.00,-207022.00,898857.00,-197863.00,700994.00,0.00,0.00,0.00,0.00,117414.00,0.00,1417254.00,0.00,5213410.00,0.00,0.00,0.00,0.00,1787061.00,29929.00,4391421.00,0.00,397311.00,0.00,6605722.00,11819132.00,0.00,0.00,1212145.00,0.00,0.00,1212145.00,2753385.00,11819132.00,0.00,1424392.00,0.00,116848.00,0.00,0.00,0.00,0.00,1541240.00,0.00,0.00,286740.00,8659602.00,0.00,0.00,0.00,9065747.00,4416674.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,89169.00,0.00,0.00,-86919.00,0.00,0.00,0.00,0.00,-4402379.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1546085.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Audited - Signed,CONSTRUCTION,South Africa,2013-02-28,0,2013-02-28,2013,2,9,28
1,1664.00,179.56,21321504.00,-15975706.00,5345798.00,-7587234.00,1083169.00,-1158267.00,0.00,-1158267.00,0.00,0.00,-196896.00,-1355163.00,0.00,-1355163.00,0.00,0.00,0.00,0.00,188658.00,0.00,5100.00,0.00,2024608.00,0.00,0.00,0.00,0.00,3614828.00,2680206.00,3949971.00,0.00,591294.00,0.00,10836299.00,12860907.00,0.00,0.00,2220239.00,0.00,0.00,2220239.00,4467976.00,12860907.00,0.00,2231289.00,0.00,0.00,0.00,0.00,16448.00,0.00,2247737.00,0.00,0.00,286740.00,7958608.00,0.00,0.00,0.00,8392931.00,8387831.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-115816.00,0.00,0.00,2680206.00,0.00,0.00,0.00,0.00,1152319.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2061712.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Audited - Signed,CONSTRUCTION,South Africa,2012-02-29,0,2012-02-29,2012,2,9,29
2,2517.00,179.56,162443991.00,-151993475.00,10450516.00,-9830040.00,277847.00,898323.00,0.00,898323.00,22666.00,0.00,-743958.00,177031.00,28770.00,205801.00,0.00,0.00,0.00,0.00,97

### Saving Processed Data to CSV

The cleaned dataset is saved as a CSV file for downstream usage. The following code saves the processed data to the `data` directory


In [11]:
df_new.to_csv('../src/data/cleaned_financial_data.csv', index=False)